In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image
from time import time

import openslide
from PIL import ImageChops
from tqdm import tqdm
from joblib import Parallel, delayed
import time
from IPython.display import display
from matplotlib.pyplot import imshow

import scipy.io as sio

resnet101_input_size = 256

# Get the tiles of the search areas on WSI that Dr. marked.

In [ ]:
# bound_x = int(img.properties["openslide.bounds-x"])
# bound_y = int(img.properties["openslide.bounds-y"])
# bound_w = int(img.properties["openslide.bounds-width"])
# bound_h = int(img.properties["openslide.bounds-height"])

# tiles -> hovernet inference -> find potential CB cell(s) -> save each one as ROI (save the image of the tile also)

searchAreas = dict() # dict of list of bbox of search area

# CB-001-1
tmp = []
tmp.append((227594,79236,228447,79868)) # (xmin,ymin,xmax,ymax)
tmp.append((224456,81643,224803,81924))
tmp.append((222253,88223,222669,88574))
tmp.append((223866,90258,224916,91558))
tmp.append((245297,87580,246037,88043))
tmp.append((241622,88999,245862,90670))
tmp.append((246425,91618,246741,91900))
searchAreas["CB-001-1"] = tmp

# CB-001-2
tmp = []
tmp.append((25086,95182,25324,95409))
tmp.append((24328,105451,25434,106222))
tmp.append((22093,109242,22982,110040))
tmp.append((22911,113051,23591,113592))
tmp.append((26524,110554,27575,111374))
tmp.append((23418,114431,24257,115037))
tmp.append((24011,116295,24300,116571))
searchAreas["CB-001-2"] = tmp

# CB-001-3
tmp = []
tmp.append((181985,141258,184498,141873)) # 184498-181985 = 2513    141973-141258 = 715
tmp.append((178292,146264,179109,146865))
tmp.append((185366,147100,185592,147301))
tmp.append((196716,134240,197734,134986))
tmp.append((197639,135809,198310,136307))
tmp.append((200051,137044,200579,137408))
tmp.append((199197,138915,201446,139920))
tmp.append((200328,141060,201030,141631))
tmp.append((200192,146371,200910,146831))
searchAreas["CB-001-3"] = tmp

# CB=002-1
tmp = []
tmp.append((61095,122140,61538,122561))
tmp.append((59733,123694,60672,123978))
tmp.append((85675,101608,86121,101901))
tmp.append((83754,106213,84264,106548))
tmp.append((85443,133843,87912,135462))
searchAreas["CB-002-1"] = tmp

num_img = 1

for filename in searchAreas.keys():
    img = openslide.OpenSlide('../data/' + filename + '.mrxs')
    for bbox in searchAreas[filename]:
        xmin,ymin,xmax,ymax= bbox
        num_tiles_row = (ymax-ymin) // resnet101_input_size
        num_tiles_col = (xmax-xmin) // resnet101_input_size
        for row in range(0, num_tiles_row): 
            for col in range(0, num_tiles_col):
                coord = (col*resnet101_input_size + xmin, row*resnet101_input_size + ymin)
                tile = img.read_region((coord[0],coord[1]), 0, (resnet101_input_size,resnet101_input_size)) # PIL image
                tile.save('/centroblast/tmp/image/' + str(num_img).zfill(4) + '.png')
                num_img += 1
        

In [7]:
def getROIfromMat(path):
    filename = path[-13:] # path ends with imagexxxx.mat
    filename = filename[:-4]
    img = Image.open('/centroblast/tmp/image/' + filename + ".png")

    um_per_px = 0.1538
    size_threshold = 5.13
    mat = sio.loadmat(path)
    # print(mat)
    mask = mat["inst_map"]
    # mask = cv2.resize(mask,(160,160))
    # mask = cv2.resize(mask,(256,256))
    num_components = np.max(mask) # number of cells detected for this image

    mask4eachComponent = np.zeros([resnet101_input_size,resnet101_input_size,num_components+1],dtype=np.uint8) # channel 1 for object 1, channel 2 for object 2, etc. channel 0 unused
    for obj in range(1,num_components+1):
        mask4eachComponent[:,:,obj] = (mask == obj)*255

    cb_detected = []
    
    def isCellCB(contours):
        if len(contours) == 0:
            return False
        area = cv2.contourArea(contours[0])
        equi_diameter = np.sqrt(4*area/np.pi)*um_per_px
        # check ratio of major/miner axis (how round it is)
        rect = cv2.boundingRect(contours[0])
        aspectRatio = rect[2] / rect[3]

        if equi_diameter > size_threshold and (0.7 <= aspectRatio and aspectRatio <= 1.3):
            return True
        else:
            return False

    counter = 1
    for obj in range(1,num_components+1):
        ret,thresh = cv2.threshold(mask4eachComponent[:,:,obj],127,255,0)
        contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        if isCellCB(contours):
            # find the centroid 
            M = cv2.moments(contours[0])
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cb_detected.append((obj,cX,cY,mask4eachComponent[:,:,obj]))
            bbox = (cX-45,cY-45,cX+45,cY+45)
            roi = img.crop(bbox)
            roi.save('/centroblast/tmp/roi_extracted/'+filename+'_ROI'+str(counter).zfill(4)+'_'+str(bbox[0]).zfill(3)+'_'+str(bbox[1]).zfill(3)+'_'+str(bbox[2]).zfill(3)+'_'+str(bbox[3]).zfill(3)+'.png')
            counter += 1


    return cb_detected, mask


In [8]:
file_lst = os.listdir("/centroblast/tmp/output/mat")
for f in file_lst:
    f = "/centroblast/tmp/output/mat/" + f
    getROIfromMat(f)

# Play Area

In [ ]:
cbs, mask = getROIfromMat("../tmp/mat/image0103.mat")

In [ ]:
plt.imshow(cbs[0][3], cmap='gray')
plt.savefig("tmp.png")

In [ ]:
print(np.unique(mask))

In [ ]:
tiles[1]

In [ ]:
# find all-white tile

In [ ]:
tile1 = img.read_region(tiles[20000], 0, (224,224))
tile2 = img.read_region((0,0), 0, (224,224))
tile3 = img.read_region((175413,101819), 0, (224,224))

In [ ]:
imshow(np.array(tile1))

In [ ]:
np.mean(np.array(tile1)[:,:,2])

In [ ]:
imshow(np.array(tile2))

In [ ]:
np.mean(np.array(tile2)[:,:,2])

In [ ]:
imshow(np.array(tile3))

In [ ]:
np.mean(np.array(tile3)[:,:,2])

In [ ]:
np.array(tile2)[1,1,1]

In [ ]:
img.properties["openslide.bounds-x"]
img.properties["openslide.bounds-y"]
img.properties["openslide.bounds-width"]
img.properties["openslide.bounds-height"]

In [ ]:

searchAreas = dict() # dict of list of bbox of search area

# CB-001-1
tmp = []
tmp.append((227594,79236,228447,79868)) # (xmin,ymin,xmax,ymax)
tmp.append((224456,81643,224803,81924))
tmp.append((222253,88223,222669,88574))
tmp.append((223866,90258,224916,91558))
tmp.append((245297,87580,246037,88043))
tmp.append((241622,88999,245862,90670))
tmp.append((246425,91618,246741,91900))
searchAreas["CB-001-1"] = tmp

# CB-001-2
tmp = []
tmp.append((25086,95182,25324,95409))
tmp.append((24328,105451,25434,106222))
tmp.append((22093,109242,22982,110040))
tmp.append((22911,113051,23591,113592))
tmp.append((26524,110554,27575,111374))
tmp.append((23418,114431,24257,115037))
tmp.append((24011,116295,24300,116571))
searchAreas["CB-001-2"] = tmp

# CB-001-3
tmp = []
tmp.append((181985,141258,184498,141873))
tmp.append((178292,146264,179109,146865))
tmp.append((185366,147100,185592,147301))
tmp.append((196716,134240,197734,134986))
tmp.append((197639,135809,198310,136307))
tmp.append((200051,137044,200579,137408))
tmp.append((199197,138915,201446,139920))
tmp.append((200328,141060,201030,141631))
tmp.append((200192,146371,200910,146831))
searchAreas["CB-001-3"] = tmp

# CB=002-1
tmp = []
tmp.append((61095,122140,61538,122561))
tmp.append((59733,123694,60672,123978))
tmp.append((85675,101608,86121,101901))
tmp.append((83754,106213,84264,106548))
tmp.append((85443,133843,87912,135462))
searchAreas["CB-002-1"] = tmp

In [ ]:
print(searchAreas)

In [ ]:
for filename in searchAreas.keys():
    print(filename)

In [ ]:
searchAreas["CB-001-1"]

In [ ]:
s = "../tmp/mat/image0103.mat"
s[-13:]

In [ ]:
img = Image.open("/home/users/v-sumeth/cell_segment/hover_net/roi_extracted/image0103_ROI0001_048_-24_138_066.png")
img_arr = np.array(img)
print(img_arr.shape)
plt.imshow(img_arr[...,:3])

In [ ]:
arr = np.load("../tmp/test.npy")

In [ ]:
plt.imshow(arr[0,:,:,3])

In [ ]:
X = arr[0,:,:,3]

In [ ]:
from skimage import measure
from skimage import filters
import matplotlib.pyplot as plt
import numpy as np

n = 12
l = 256
np.random.seed(1)
im = np.zeros((l, l))
points = l * np.random.random((2, n ** 2))
im[(points[0]).astype(np.int), (points[1]).astype(np.int)] = 1
im = filters.gaussian(im, sigma= l / (4. * n))
blobs = im > 0.7 * im.mean()

all_labels = measure.label(blobs)
blobs_labels = measure.label(blobs, background=0)

plt.figure(figsize=(9, 3.5))
plt.subplot(131)
plt.imshow(blobs, cmap='gray')
plt.axis('off')
plt.subplot(132)
plt.imshow(all_labels, cmap='nipy_spectral')
plt.axis('off')
plt.subplot(133)
plt.imshow(blobs_labels, cmap='nipy_spectral')
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
whos

In [ ]:
def test_func(path):
    filename = path[-13:] # path ends with imagexxxx.mat
    filename = filename[:-4]
    img = Image.open('/home/users/v-sumeth/cell_segment/hover_net/tiles_extracted/' + filename + ".png")

    mat = sio.loadmat(path)
    mask = mat["inst_map"]
    plt.imshow(mask)
    mask2 = cv2.resize(mask/255.0, (160,160), interpolation = cv2.INTER_AREA)
    mask2 = cv2.resize(mask2,(256,256))
    mask2 = (mask2*255).astype(np.uint8)
    plt.imshow(mask2)

In [ ]:
test_func("../tmp/mat/image0103.mat")

In [ ]:
whos